In [1]:
from create_overview import ResultAggregator
from copy import deepcopy
import re
from ast import literal_eval
from statistics import mean
import numpy as np
from collections import defaultdict
import pandas as pd

# 1. Starting

- There are two ways to start the code
 - You can either directly fetch the data from the project from [wandb](https://wandb.ai/site) or
 - you can read the data from a pre-existing csv file
- Note: Whenever you fetch the data directly from wandb, the data is automatically stored in a csv file called ```current_wandb_results_unprocessed.csv```. This measurement is taken, because downloading the data from wandb can take a while and sometimes even be interrupted due to connection issues (```Read timed out.``` etc.). Therefore, having a csv file can speed up the data reading.
- Initializing the object can happen in two ways, depending on whether you want to fetch the data from wandb or whether you want to read it from an existing csv file
- To fetch the data directly from wand to you need to pass a valid wand api key:

```
ra = ResultAggregator(wandb_api_key={WAND_API_KEY})

```
- To use the data from an existing csv file, you need to pass the path to the csv file (in the example below, the path to the automatically generated csv file is given):

```
ra = ResultAggregator(path_to_csv_export="current_wandb_results_unprocessed.csv")

```

- Optionally, you can set the argument ```verbose_logging```to ```False```. This might be usefule, to avoid overwhelming logging messages either in the terminal or in a notebook cell. The logs are saved in a log file in the log directory anyway.
- Passing the argument ```verbose_logging``` can be done like this:

```
ra = ResultAggregator(wandb_api_key={WAND_API_KEY}, verbose_logging=False)

```

```
ra = ResultAggregator(path_to_csv_export="current_wandb_results_unprocessed.csv", verbose_logging=False)

```

- It is important to note that all overviews and aggregates evolves around the score `macro-f1`. But you can overwrite this by passing `score` with the name of the score you are interested in, like this:

```
ra = ResultAggregator(path_to_csv_export="current_wandb_results_unprocessed.csv", score='micro-f1')

```
 

In [2]:
wandb_api_key = "3b7e4cee8259b9c6522aa3adef3833e6c109f000"
ra = ResultAggregator(wandb_api_key=wandb_api_key, verbose_logging=False)
#ra = ResultAggregator(path_to_csv_export="current_wandb_results_unprocessed.csv", verbose_logging=False)


wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integ

CommError: HTTPSConnectionPool(host='api.wandb.ai', port=443): Read timed out. (read timeout=9)

# 2. General information

- When fetching the data from wandb, the data is processed and saved to a dataframe called `results` which is an object variable
- **In contains only runs that are finished. Failed or crashed runs on wandb have been removed**
- You can access the processed data like this `ra.results`
- This data will be used to calculate the scores
- All overviews with the calculated aggregate scores will be saved to dataframes as well, which can be accessed likewise



In [ ]:
ra.results.head()

# 2. Overviews

- The code was writing while some runs for certain had not been finished 
- Therefore, you can specify whether the generated overviews should consider only taks whose run had been completed entirely. This is automatically done by setting ```only_completed_tasks```to ```True``` as a default value.
- In case you want to see interim results, including tasks that have not been completed yet, you can set ```only_completed_tasks```to ```False``` when intitiliazing:

```
ra = ResultAggregator(path_to_csv_export="current_wandb_results_unprocessed.csv", only_completed_tasks=False)

```

- The code will create a bunch of different overviews that will be described in the following

## 2.1. Incomplete tasks

- If you want to generate an overview of incomplete tasks you run the following: `ra.check_seed_per_task()`
- This method will check if all task have the required number of finished runs with the seeds 1,2,3


In [ ]:
seed_check = ra.check_seed_per_task()
seed_check.head(15)

##  2.2. Overviews along scores

- The method ```create_overview_of_results_per_seed()``` will generate an overview of the required score per seed and give a mean value. If not specified otherwise, the score defined during initializing will be used, i.e. `macro-f1`

In [ ]:
ra.create_overview_of_results_per_seed()

- If you want to to generate an overview for another score, let's say `micro-f1`, you can pass the name of the score as a variable to the function


In [ ]:
ra.create_overview_of_results_per_seed(score='micro-f1')

## 2.3 Report
- As a shortcut, you can use the method `create_report()`
- It will run `create_overview_of_results_per_seed` over different scores and give an overview of missing runs.
- The overviews will be saved in an excel file called `report.xlsx` in the directory `results`

In [ ]:
ra.create_report()

- When running `create_report()` the generated overviews will be stored as dataframes in object variables that can be directly accessed.
- Currently, the variables are:
  - seed_check
  - macro_f1_overview
  - micro_f1_overview
  - weighted_f1_overview
  - accuracy_normalized_overview
  
- You can use them for visualization purposes

In [ ]:
ra.macro_f1_overview[['finetuning_task', 'mean_over_seeds']].groupby('finetuning_task').mean().sort_values('mean_over_seeds').plot.bar()


In [ ]:
ra.accuracy_normalized_overview[['finetuning_task', 'mean_over_seeds']].groupby('finetuning_task').mean().sort_values('mean_over_seeds').plot.bar()

### 2.4. Aggregated scores

### 2.4.1. Language aggregate

`DEFINITION`: *"1. Average over configurations (e.g. judgment and unanimity) inside datasets, 2. Average over Datasets, 3. Average over languages"*

- The aggregated score along languages (the language aggregate score) can be generated with the method `get_language_aggregated_score()`
  - Since, the default score is `macro-f1`, the aggregate score is based on macro-f1. You can change this by defining another score when initializing the object (see description above).
- The result dataframe is stored in the variable `language_aggregated_score`
- When calling the method, the resulting dataframe is saved to a csv and xlsx file, called `language_aggregated_scores` in the results folder

In [ ]:
ra.get_language_aggregated_score()
ra.language_aggregated_score.head()

- You can restrict the list of tasks you want to consider by passing a list of task names 

In [ ]:
ra.get_language_aggregated_score(task_constraint=['german_argument_mining'])
ra.language_aggregated_score.head(50)

### 2.4.2. Config aggregate

- The aggregated score along configs (the task config score) can be generated with the method `get_config_aggregated_score()`
  - Since, the default score is `macro-f1`, the aggregate score is based on macro-f1. You can change this by defining another score when initializing the object (see description above).
  - Per default the method calcualtes the average over config, dataset, language. To overwrite this behavious, i.e. to calculate the avergae score not over languages, you can set average_over_language=False. The resulting values should be similar to the aggregate scores we find on wandb
- The result dataframe is stored in the variable `config_aggregated_score`
- When calling the method, the resulting dataframe is saved to a csv and xlsx file
  - Depending on whether you average over language or not the name of the file is called `config_aggregated_scores_simple` (not averaged over language) OR `config_aggregated_scores_average_over_language` (averaged over language) in the results folder
  
**The dataset aggregate score is built on top of the results of the config aggregate score**

In [ ]:
ra.get_config_aggregated_score()
ra.config_aggregated_score.head()

### 2.4.3. Dataset aggregate
`DEFINITION`: *"1. Average over languages inside configurations, 2. Average over configurations (e.g. judgment and unanimity) inside datasets, 3. Average over Datasets"*

- The aggregated score along datasets (the dataset aggregate score) can be generated with the method `get_dataset_aggregated_score()`
  - Since, the default score is `macro-f1`, the aggregate score is based on macro-f1. You can change this by defining another score when initializing the object (see description above).
  - Per default the method calcualtes the average over config, dataset, language. To overwrite this behavious, i.e. to calculate the avergae score not over languages, you can set average_over_language=False. The resulting values should be similar to the aggregate scores we find on wandb
- The result dataframe is stored in the variable `dataset_aggregated_score`
- When calling the method, the resulting dataframe is saved to a csv and xlsx file
  - Depending on whether you average over language or not the name of the file is called `dataset_aggregated_scores_simple` (not averaged over language) OR `dataset_aggregated_scores_average_over_language` (averaged over language) in the results folder

In [ ]:
ra.get_dataset_aggregated_score()
ra.dataset_aggregated_score.head()

In [ ]:
ra.get_dataset_aggregated_score(average_over_language=False)
ra.dataset_aggregated_score.head()

## 3. Additional features

### 3.1. Task constraint
- We want to calculate a task aggregate score
- It might be useful, to remove some tasks from the calculations
- This can be done by passing a list of tasks we want to consider; see the example below

In [ ]:
print('Results for all available tasks:')
print('------------------------')
ra.get_language_aggregated_score()
ra.get_dataset_aggregated_score(average_over_language=True)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

tasks_we_are_interested_in = ['mapa_fine', 'german_argument_mining']
print('Results for only selected tasks:', ', '.join(tasks_we_are_interested_in))
print('------------------------')
ra.get_language_aggregated_score(task_constraint=tasks_we_are_interested_in)
ra.get_dataset_aggregated_score(average_over_language=True, task_constraint=tasks_we_are_interested_in)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

ra.reset_list_of_available_languages()


### 3.2. Language constraint
- We want to calculate a language aggregate score
- It might be useful, to remove some languages from the calculations
- This can be done with the method `remove_languages({LANGUAGE_ID})`
- You can reset the initial number of languages with the method `reset_list_of_available_languages()`
- This might be useful, if you want to gauge the influence of certain languages on the overall scores (see example below)

In [ ]:
ra.reset_list_of_available_languages()

print('Results with all available languages:')
print('------------------------')
ra.get_language_aggregated_score()
ra.get_dataset_aggregated_score(average_over_language=True)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

ra.reset_list_of_available_languages()

ra.remove_languages('pt')

print('Results without Portuguese:')
print('------------------------')
ra.get_language_aggregated_score()
ra.get_dataset_aggregated_score(average_over_language=True)

lm = ["distilbert-base-multilingual-cased", "microsoft/Multilingual-MiniLM-L12-H384", "xlm-roberta-base", "microsoft/mdeberta-v3-base", "xlm-roberta-large"]

for x in lm:
    print(x)
    print('\t','language aggregated: ',ra.language_aggregated_score.at[x,'aggregated_score'])
    print('\t','dataset aggregated: ', ra.dataset_aggregated_score.at[x,'aggregated_score'])
    print('-----------------------------------')

print('\n##########################\n')

ra.reset_list_of_available_languages()


## 4. Analysis

### 4.1. Prepare tables for LaTeX

- In order for this to work, you need to make the following imports in your LaTeX document

```
\usepackage{rotating}
\usepackage{adjustbox}
```

In [ ]:
# We round all scores (macro-F1 values)

def round_scores(score):
    if type(score)==float:
        return round(score,2)
    else:
        return score

In [ ]:
# Formatting so that the large tables fit to one page
# See: https://tex.stackexchange.com/questions/121155/how-to-adjust-a-table-to-fit-on-page 

pre_formatting = ""
pre_formatting = pre_formatting + "\\begin{sidewaystable}\n"
#pre_formatting = pre_formatting + "\centering\n"
pre_formatting = pre_formatting + "\\begin{adjustbox}{width=1\\textwidth}\n"
pre_formatting = pre_formatting + "\medium\n"
#pre_formatting = pre_formatting + "\caption{Language aggregates}\n"
#pre_formatting = pre_formatting + "\label{tab:language aggregates}\n"


after_formatting = ""
after_formatting = after_formatting + "\end{adjustbox}\n"
after_formatting = after_formatting + "\caption{Language aggregate scores}"
after_formatting = after_formatting + "\label{T4:language_aggregate}"
after_formatting = after_formatting + "\end{sidewaystable}\n"
#pre_formatting = pre_formatting + "\n"

ra.get_language_aggregated_score()
language_aggregated_score = ra.language_aggregated_score
language_aggregated_score = language_aggregated_score.applymap(round_scores)
language_aggregated_score.rename(columns={'aggregated_score':'aggregated score'}, inplace=True)
language_aggregated_score_for_latex = pre_formatting + language_aggregated_score.to_latex() + after_formatting
print(language_aggregated_score_for_latex)

In [ ]:
# Formatting so that the large tables fit to one page
# See: https://tex.stackexchange.com/questions/121155/how-to-adjust-a-table-to-fit-on-page 

pre_formatting = ""
pre_formatting = pre_formatting + "\\begin{sidewaystable}\n"
#pre_formatting = pre_formatting + "\centering\n"
pre_formatting = pre_formatting + "\\begin{adjustbox}{width=1\\textwidth}\n"
pre_formatting = pre_formatting + "\medium\n"


after_formatting = ""
after_formatting = after_formatting + "\end{adjustbox}\n"
after_formatting = after_formatting + "\caption{Dataset aggregate scores}"
after_formatting = after_formatting + "\label{T5:dataset_aggregate}"
after_formatting = after_formatting + "\end{sidewaystable}\n"
#pre_formatting = pre_formatting + "\n"

ra.get_dataset_aggregated_score()
dataset_aggregated_score = ra.dataset_aggregated_score
dataset_aggregated_score = dataset_aggregated_score.applymap(round_scores)
dataset_aggregated_score.rename(columns={'aggregated_score':'aggregated score'}, inplace=True)
dataset_aggregated_score_for_latex = pre_formatting + dataset_aggregated_score.to_latex() + after_formatting
print(dataset_aggregated_score_for_latex)